# 03 - Modelo SVM (estilo completo, siguiendo pipeline)

Notebook preparado para usar el archivo `train_preprocesado_completo.csv` generado por el pipeline `02 - preprocesado.ipynb`. Usa `RENDIMIENTO_GLOBAL` como target. Este notebook está diseñado para verse tan completo como el de tu compañero: secciones extensas, tablas, gráficos y exportación de resultados.

**Recomendación:** ejecuta antes la celda del notebook 02 para generar `train_preprocesado_completo.csv`.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import pickle

# Para que las figuras se vean en el notebook
%matplotlib inline

In [ ]:
# 1) Cargar datos preprocesados
DATA_PATH = 'train_preprocesado_completo.csv'

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"No se encontró {DATA_PATH}. Ejecuta el notebook 02 para generarlo o coloca el archivo en la carpeta.")

print('Cargando:', DATA_PATH)
df = pd.read_csv(DATA_PATH)
print('Dimensiones:', df.shape)
df.head()

In [ ]:
# 2) Información y primeras tablas
print('Columnas:', df.columns.tolist())
print('\nTipos de datos:')
print(df.dtypes.value_counts())

print('\nMuestra de estadísticas (numéricas):')
df.describe().T

In [ ]:
# 3) Análisis del target `RENDIMIENTO_GLOBAL`
target = 'RENDIMIENTO_GLOBAL'
if target not in df.columns:
    raise KeyError(f"La columna target '{target}' no está en el dataset. Columnas disponibles: {df.columns.tolist()}")

print('Valores únicos del target:', df[target].unique())
print('\nDistribución del target:')
print(df[target].value_counts(normalize=True))

plt.figure(figsize=(8,4))
sns.countplot(data=df, x=target)
plt.title('Distribución de RENDIMIENTO_GLOBAL')
plt.show()

## Archivos individuales generados por el pipeline
El pipeline 02 genera CSV individuales por cada variable categórica (si `generar_csv_individual=True`). A continuación listamos archivos `trainpreprocesado{col}.csv` que existan en la carpeta para que puedas inspeccionarlos.

In [ ]:
import glob
ind_files = sorted(glob.glob('trainpreprocesadopreprocesado*.csv') + glob.glob('trainpreprocesado*.csv') + glob.glob('train*preprocesado*.csv'))
print('Archivos encontrados (algunos patrones):')
for f in ind_files[:20]:
    print('-', f)

if not ind_files:
    print('No se encontraron CSV individuales con patrón; esto es opcional y depende del pipeline.')

## 4) Preparación de X e y
Separamos la columna `ID` (si existe) y la columna objetivo `RENDIMIENTO_GLOBAL`. Además guardamos un `ID` para poder volver a unir predicciones.

In [ ]:
df_proc = df.copy()
ID_col = None
if 'ID' in df_proc.columns:
    ID_col = df_proc['ID']
    df_proc = df_proc.drop(columns=['ID'])

# Separar X,y
X = df_proc.drop(columns=[target])
y = df_proc[target]

print('X shape:', X.shape)
print('y shape:', y.shape)
X.head()

In [ ]:
# 5) Estadísticas de variables numéricas y correlaciones (si aplica)
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
print('Número de columnas numéricas:', len(num_cols))

if len(num_cols) > 0:
    display(X[num_cols].describe().T)
    plt.figure(figsize=(10,8))
    sns.heatmap(X[num_cols].corr(), annot=False, cmap='RdBu', center=0)
    plt.title('Mapa de correlación (numéricas)')
    plt.show()
else:
    print('No se detectaron columnas numéricas.')

## 6) Split Train/Test
Usamos `train_test_split` estratificado y mostramos tamaños y primeros registros.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

print('Train:', X_train.shape, 'Test:', X_test.shape)

# Mostrar primeras filas de train
train_preview = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_preview.head(10)

## 7) Estandarización y Pipeline
Para SVM es importante estandarizar las variables numéricas. Construimos un pipeline con `StandardScaler` + `SVC`.

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, random_state=42))
])

param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__gamma': ['scale', 'auto', 0.01],
    'svc__kernel': ['rbf']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('Pipeline y grid listos. Ejecutando GridSearchCV...')

In [ ]:
grid = GridSearchCV(pipeline, param_grid, cv=cv, scoring='f1_weighted', n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

print('Mejor score (CV):', grid.best_score_)
print('Mejores parámetros:', grid.best_params_)

best_model = grid.best_estimator_

## 8) Evaluación en Test
Mostramos métricas clásicas y matriz de confusión. Además calculamos ROC AUC si es binario.

In [ ]:
y_pred = best_model.predict(X_test)

y_acc = accuracy_score(y_test, y_pred)
y_f1 = f1_score(y_test, y_pred, average='weighted')

print('Accuracy:', y_acc)
print('F1 (weighted):', y_f1)
print('\nClassification report:\n')
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix:\n', cm)

In [ ]:
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de confusión - Test')
plt.ylabel('Actual')
plt.xlabel('Predicho')
plt.show()

In [ ]:
if len(y.unique()) == 2:
    y_proba = best_model.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_proba)
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    print('ROC AUC:', auc)
    plt.figure()
    plt.plot(fpr, tpr)
    plt.plot([0,1],[0,1],'--')
    plt.title('ROC Curve')
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.show()
else:
    print('Problema multiclase: no se muestra ROC AUC automáticamente.')

## 9) Guardar modelo y objetos
Guardamos el modelo final y el scaler en la carpeta `outputs/`.

In [ ]:
os.makedirs('outputs', exist_ok=True)
with open('outputs/modelo_03_SVM.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Guardar métricas
metrics = {'accuracy': float(y_acc), 'f1_weighted': float(y_f1), 'best_params': grid.best_params_}
import json
with open('outputs/metrics_modelo_03_SVM.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print('Model y métricas guardados en outputs/.')

In [ ]:
print('\nArchivos en outputs/')
import glob
for f in glob.glob('outputs/*'):
    print('-', f)

### Notas finales
- Si quieres que el notebook use el pipeline 02 directamente, podríamos importar `preprocesar_pipeline` y generar el `train_preprocesado_completo.csv` desde aquí.
- Si deseas que en vez de GridSearchCV usemos RandomizedSearchCV o ajustes de hiperparámetros más amplios, lo modifico.

Listo — este notebook está preparado para ser visualmente largo, con tablas y gráficos, listo para presentación.